

# Proyecto

### Equipo:

- Sebastian Avendaño
- Felipe Urrutia

- \<Nombre de usuarios en Codalab\>

- \<Nombre del Equipo en Codalab\>

### Link de repositorio de GitHub: https://github.com/furrutiav/lab-mds-2022



## 0. Librerías Utilizadas

In [2]:
# Carga y Preparación de los datos

import pickle
import pandas as pd
import numpy as np
!pip install pyarrow

# EDA

import plotly.express as px


## 1. Introducción

El objetivo de este proyecto consiste en ...

Los datos que proveen es un dataset con X ejemplos que describen una observación de ... 
Son N atributos y la variables objetivos son de tipo ... y ...

La primera tarea se evalua en base a la métrica ... ya que esta permite medir ...

Nuestra propuesta para resolver el problema consistió en modelo de clasificación basado en...

Nuestro modelo cumplió/no cumplió las expectativas...

---
## 2. Preparación de los datos

#### Carga y Preparación de los Datos

- Cargar los datos con Pandas y fusionar por `id`.
- Eliminar columnas `'poster_path'`, `'backdrop_path'`, `'recommendations'`.
- Filtrar ejemplos con `revenue` igual a 0.
- Filtrar ejemplos con `release_date` y `runtime` nulos.
- Convertir fechas de release_date a `pd.DateTime`.
- Conservar solo los ejemplos con `status` `"Released"`.
- Rellenar valores nulos categóricos y de texto con `''`.
- Discretizar `vote_average` a los siguientes bins y guardar los resultados en la columna `label`: 
  - (0, 5]: `'Negative'`
  - (5, 6]: `'Mixed'`
  - (6, 7]: `'Mostly Positive'`
  - (7, 8]: `'Positive'`
  - (8, 10]: `'Very Positive'`
- Eliminar la columna `vote_average` e `id`
- Renombrar la columna `revenue` por `target`.

Cargar los datos con Pandas

In [3]:
# datos: train_numerical_features.parquet y train_text_features.parquet
train_numerical_features = pd.read_parquet('train_numerical_features.parquet').set_index("id")
train_text_features = pd.read_parquet('train_text_features.parquet').set_index("id")

Fusionar por id

In [4]:
df = pd.concat([train_numerical_features, train_text_features], axis=1)

Eliminar columnas duplicadas

In [5]:
df = df.T.drop_duplicates().T

Eliminar columnas 'poster_path', 'backdrop_path', 'recommendations'

In [6]:
df = df.drop(columns=['poster_path', 'backdrop_path', 'recommendations'])

Filtrar ejemplos con revenue igual a 0.

In [7]:
df = df[df["revenue"]>0]

Filtrar ejemplos con release_date y runtime nulos.

In [8]:
df["release_date"].isna().sum()

0

In [9]:
df["runtime"].isna().sum()

0

Convertir fechas de release_date a pd.DateTime

In [10]:
df["release_date"] = df["release_date"].apply(pd.to_datetime)

Conservar solo los ejemplos con status "Released"

In [11]:
df = df[df["status"] == "Released"]

Rellenar valores nulos categóricos y de texto con ' '

In [12]:
df[train_text_features.columns.tolist()] = df[train_text_features.columns.tolist()].fillna("")

Discretizar vote_average a los siguientes bins y guardar los resultados en la columna label:

    (0, 5]: 'Negative'
    (5, 6]: 'Mixed'
    (6, 7]: 'Mostly Positive'
    (7, 8]: 'Positive'
    (8, 10]: 'Very Positive'


In [13]:
df["label"] = pd.cut(df['vote_average'], 
       bins=[0,5,6,7,8,10], 
       labels=["Negative", "Mixed", "Mostly Positive", "Positive", "Very Positive"])

Eliminar la columna vote_average e id

In [14]:
df = df.drop(columns="vote_average")
df = df.reset_index(drop=True)

Renombrar la columna revenue por target

In [15]:
df = df.rename(columns={"revenue": "target"})

Cargamos los clusters

In [23]:
clusters_BERT = pickle.load(open("clusters_BERT.pickle", "rb"))

In [30]:
df = df.merge(clusters_BERT, how="left", on="title")
df

,title,budget,target,runtime,status,tagline,credits,genres,original_language,overview,production_companies,release_date,keywords,label,clusters_keywords,clusters_overview,clusters_tagline
0,Fantastic Beasts: The Secrets of Dumbledore,200000000.0,400000000.0,142.0,Released,Return to the magic.,Jude Law-Eddie Redmayne-Mads Mikkelsen-Ezra Mi...,Fantasy-Adventure-Action,en,Professor Albus Dumbledore knows the powerful ...,Warner Bros. Pictures-Heyday Films,2022-04-06,magic-curse-fantasy world-wizard-magical creat...,Mostly Positive,8,14,11
1,Sonic the Hedgehog 2,110000000.0,393000000.0,122.0,Released,Welcome to the next level.,James Marsden-Ben Schwartz-Tika Sumpter-Natash...,Action-Adventure-Family-Comedy,en,After settling in Green Hills Sonic is eager t...,Original Film-Blur Studio-Marza Animation Plan...,2022-03-30,sequel-based on video game-hedgehog-live actio...,Positive,7,14,11
2,The Lost City,74000000.0,164289828.0,112.0,Released,The adventure is real. The heroes are not.,Sandra Bullock-Channing Tatum-Daniel Radcliffe...,Action-Adventure-Comedy,en,A reclusive romance novelist was sure nothing ...,Paramount-Fortis Films-3dot Productions-Exhibi...,2022-03-24,duringcreditsstinger,Mostly Positive,10,13,13
3,Morbius,75000000.0,161000000.0,105.0,Released,A new Marvel legend arrives.,Jared Leto-Matt Smith-Adria Arjona-Jared Harri...,Action-Science Fiction-Fantasy,en,Dangerously ill with a rare blood disorder and...,Columbia Pictures-Avi Arad Productions-Matt To...,2022-03-30,vampire-based on comic,Mostly Positive,7,10,4
4,Uncharted,120000000.0,400780000.0,116.0,Released,Fortune favors the bold.,Tom Holland-Mark Wahlberg-Sophia Ali-Tati Gabr...,Action-Adventure,en,A young street-smart Nathan Drake and his wise...,Columbia Pictures-Atlas Entertainment-PlayStat...,2022-02-10,treasure-treasure hunt-based on video game-dlb,Positive,14,14,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,Amistad,36000000.0,44229441.0,155.0,Released,Freedom is not given. It is our right at birth...,Morgan Freeman-Nigel Hawthorne-Anthony Hopkins...,Drama-History-Mystery,en,In 1839 the slave ship Amistad set sail from C...,DreamWorks Pictures,1997-12-10,cuba-mutiny-slavery-sentence-historical figure...,Mostly Positive,13,0,9
6447,Home Alone,18000000.0,476684675.0,103.0,Released,A family comedy without the family.,Macaulay Culkin-Joe Pesci-Daniel Stern-John He...,Comedy-Family,en,Eight-year-old Kevin McCallister makes the mos...,Hughes Entertainment-20th Century Fox,1990-11-16,holiday-burglar-slapstick-little boy-family re...,Positive,9,7,13
6448,Ip Man: The Final Fight,0.0,3967001.0,100.0,Released,,Anthony Wong-Anita Yuen-Gillian Chung-Jordan C...,Action-Drama,cn,In postwar Hong Kong legendary Wing Chun grand...,Emperor Motion Pictures-Cinemasia-National Art...,2013-03-22,biography,Mostly Positive,5,1,-1
6449,A Rainy Day in New York,25000000.0,23800000.0,92.0,Released,Love In Spring.,Timothée Chalamet-Elle Fanning-Selena Gomez-Ju...,Comedy-Romance,en,Two young people arrive in New York to spend a...,Gravier Productions-Perdido Productions-FilmNa...,2019-07-26,new york city,Mostly Positive,5,9,13


---

## 3. Preprocesamiento, Holdout y Feature Engineering

#### ColumnTransformer y Holdout

*Esta sección consiste en generar los distintos pasos para preparar sus datos con el fin de luego poder crear su modelo.*

Generar un ColumnTransformer que:

- Preprocese datos categóricos y ordinales.
- Escale/estandarice datos numéricos.
- Codifique texto.

Luego, pruebe las transformaciones utilizando `fit_transform` y `get_feature_names out`.

Posteriormente, ejecute un Holdout que le permita más adelante evaluar los modelos. **Recuerde eliminar los target y las labels del dataset antes de dividirlo**.

In [31]:
## Código Holdout
from sklearn.model_selection import train_test_split

In [32]:
X_clf, y_clf = df.drop(columns=["target", "label"]), df["label"]
X_reg, y_reg = df.drop(columns=["target", "label"]), df["target"]

In [33]:
X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(X_clf, y_clf, test_size=0.20, random_state=23)
X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(X_reg, y_reg, test_size=0.20, random_state=23)

In [35]:
!pip install nltk
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [36]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, FunctionTransformer

In [37]:
## Código ColumnTransformer
atributos_minmax = [
    "budget"
]
atributos_st = [
    "runtime"
]
atributos_onehot = [
    "original_language"
]

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc.lower())
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]
    
class SplitTokenizer:
    def __init__(self, char="-"):
        self.char = char
    def __call__(self, doc):
        return doc.replace(" ", "_").split(self.char)

ct = ColumnTransformer([
    ("OneHot", OneHotEncoder(handle_unknown="ignore"), atributos_onehot),
    ("MinMax", MinMaxScaler(), atributos_minmax),
    ("Standard", StandardScaler(), atributos_st),
    ("BOW", CountVectorizer(tokenizer= StemmerTokenizer()), "overview"),
    ("OneHot_split_genres", CountVectorizer(tokenizer= SplitTokenizer()), "genres"),
    ("OneHot_split_credits", CountVectorizer(tokenizer= SplitTokenizer()), "credits"),
    ("OneHot_split_production_companies", CountVectorizer(tokenizer= SplitTokenizer()), "production_companies"),
    ("OneHot_split_keywords", CountVectorizer(tokenizer= SplitTokenizer()), "keywords"),
])

#### Feature Engineering

Adicionalmente puede generar una nueva transformación que genere nuevas features y que se aplique antes del ColumnTransformer dentro del pipeline de los modelos. Investigar [`FunctionTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html) para ver como implementar una transformación a partir de una función que tome un dataframe y entregue uno distinto en la salida.

- Encodear ciclicamente los meses/días de las fechas de lanzamiento.
- Contar cuantas veces aparecen en las peliculas ciertos personajes célebres.
- Indicar si la pelicula es de una productora famosa o no.
- Agrupar distintas keywords en categorías más generales.
- Generar ratios con las variables numericas del dataset (como duración de la película/presupuesto).
- Contar los diferentes generos similares que posee una pelicula.
- Extraer vectores desde los overviews de las peliculas.
- Contar el número de actores/productoras/géneros.
- Etc... Usen su creatividad!

Nuevamente, recuerde no utilizar ni los targets ni las labels para generar nuevas features.

Nota: Este último paso no es requisito pero puede catapultarlos a la cima del tablero de las competencias.

In [5]:
## Código Feature Engineering (Opcional)

```
Comentarios
```

---

## 4. Clasificación

### 4.1 Dummy y Baseline

*En esta sección crearán el modelo más básico posible que resuelva el problema. La idea de este modelo usarlo como comparación para que en el siguiente paso lo puedan mejorar.*

- Generar un modelo Dummy con estrategia estratificada que les permita comparar más adelante si su baseline de clasificación es mejor que el azar.
- Generar un pipeline para la clasificación con un clasificador relativamente sencillo a la salida (a su elección, recomendado: arbol de decisión).
- Evaluar ambos modelos según las métricas de evaluación y reportar.

In [105]:
from sklearn.dummy import DummyClassifier

In [116]:
## Código Dummy
dummy_clf = DummyClassifier(strategy="stratified", random_state=0)
dummy_clf.fit(X_clf_train, y_clf_train)

y_pred_dummy = dummy_clf.predict(X_clf_test)

In [125]:
from sklearn.pipeline import Pipeline

In [126]:
from sklearn.ensemble import RandomForestClassifier

In [143]:
## Código Clasificador
pipe_clf_baseline = Pipeline(
    steps=[
        ("ColumnTransformer", ct),
        ("clf", RandomForestClassifier(class_weight="balanced", max_depth=3, random_state=0))
    ]
)

In [144]:
%%time 
pipe_clf_baseline.fit(X_clf_train, y_clf_train)

Wall time: 7.58 s


Pipeline(steps=[('ColumnTransformer',
                 ColumnTransformer(transformers=[('OneHot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['original_language']),
                                                 ('MinMax', MinMaxScaler(),
                                                  ['budget']),
                                                 ('Standard', StandardScaler(),
                                                  ['runtime']),
                                                 ('BOW',
                                                  CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x0000025F4E6D79A0>),
                                                  'overview'),
                                                 ('OneHot_split_genres',
                                                  CountVe...
                                                  'credits'),
    

In [145]:
%%time 
y_pred_baseline = pipe_clf_baseline.predict(X_clf_test)

Wall time: 1.76 s


In [119]:
from sklearn.metrics import classification_report

In [136]:
## Código Comparación de métricas
print(
    "Clasificador Dummy\n"+classification_report(y_clf_test, y_pred_dummy)
)

Clasificador Dummy
                 precision    recall  f1-score   support

          Mixed       0.20      0.23      0.21       252
Mostly Positive       0.47      0.43      0.45       621
       Negative       0.05      0.05      0.05        37
       Positive       0.25      0.25      0.25       343
  Very Positive       0.04      0.05      0.05        38

       accuracy                           0.32      1291
      macro avg       0.20      0.20      0.20      1291
   weighted avg       0.33      0.32      0.33      1291



In [146]:
print(
    "Clasificador baseline: RF\n"+classification_report(y_clf_test, y_pred_baseline)
)

Clasificador baseline: RF
                 precision    recall  f1-score   support

          Mixed       0.31      0.54      0.39       252
Mostly Positive       0.55      0.25      0.34       621
       Negative       0.03      0.03      0.03        37
       Positive       0.38      0.55      0.45       343
  Very Positive       0.12      0.13      0.13        38

       accuracy                           0.37      1291
      macro avg       0.28      0.30      0.27      1291
   weighted avg       0.43      0.37      0.36      1291



```
Justificación
```

---

### 4.2 Búsqueda del mejor modelo de Clasificación


In [1]:
### Código GridSearch

In [ ]:
### Código Predicción de datos de la competencia aquí

```
Justificación Aquí
```

---

## 5. Conclusiones Clasificación

Conclusiones...



---

<br>

### Anexo: Generación de Archivo Submit de la Competencia

Para subir los resultados obtenidos a la pagina de CodaLab utilice la función `generateFiles` entregada mas abajo. Esto es debido a que usted deberá generar archivos que respeten extrictamente el formato de CodaLab, de lo contario los resultados no se veran reflejados en la pagina de la competencia.

Para los resultados obtenidos en su modelo de clasificación y regresión, estos serán guardados en un archivo zip que contenga los archivos `predicctions_clf.txt` para la clasificación y `predicctions_rgr.clf` para la regresión. Los resultados, como se comento antes, deberan ser obtenidos en base al dataset `test.pickle` y en cada una de las lineas deberan presentar las predicciones realizadas.

Ejemplos de archivos:

- [ ] `predicctions_clf.txt`

        Mostly Positive
        Mostly Positive
        Negative
        Positive
        Negative
        Positive
        ...

- [ ] `predicctions_rgr.txt`

        16103.58
        16103.58
        16041.89
        9328.62
        107976.03
        194374.08
        ...



In [ ]:
from zipfile import ZipFile
import os

def generateFiles(predict_data, clf_pipe, rgr_pipe):
    """Genera los archivos a subir en CodaLab

    Input
    predict_data: Dataframe con los datos de entrada a predecir
    clf_pipe: pipeline del clf
    rgr_pipe: pipeline del rgr

    Ouput
    archivo de txt
    """
    y_pred_clf = clf_pipe.predict(predict_data)
    y_pred_rgr = rgr_pipe.predict(predict_data)
    
    with open('./predictions_clf.txt', 'w') as f:
        for item in y_pred_clf:
            f.write("%s\n" % item)

    with open('./predictions_rgr.txt', 'w') as f:
        for item in y_pred_rgr:
            f.write("%s\n" % item)

    with ZipFile('predictions.zip', 'w') as zipObj2:
       zipObj2.write('predictions_rgr.txt')
       zipObj2.write('predictions_clf.txt')

    os.remove("predictions_rgr.txt")
    os.remove("predictions_clf.txt")

In [ ]:
# Ejecutar función para generar el archivo de predicciones.
# perdict_data debe tener cargada los datos del text.pickle
# mientras que clf_pipe y rgr_pipe, son los pipeline de 
# clasificación y regresión respectivamente.
generateFiles(predict_data, clf_pipe, rgr_pipe)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>